In [24]:
pip install transformers

In [31]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [39]:
pip install glue

     |████████████████████████████████| 296kB 2.7MB/s 
     |████████████████████████████████| 348kB 10.2MB/s 
  Created wheel for glue: filename=glue-0.13-cp36-none-any.whl size=28282 sha256=551196f0a50f1a9f09ac042ea4437e8b993decb7a3c4b1021abc9a1c78fb15bc
  Stored in directory: /root/.cache/pip/wheels/69/09/09/e213b576abd91be00f007915e24c41aa5c897e4e48a4a28158
Successfully built glue
ERROR: flask 1.1.2 has requirement Jinja2>=2.10.1, but you'll have jinja2 2.9.6 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
  Found existing installation: Jinja2 2.11.2
    Uninstalling Jinja2-2.11.2:
      Successfully uninstalled Jinja2-2.11.2


In [44]:
import os
import re
import json
import string
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tokenizers import BertWordPieceTokenizer
from transformers import BertTokenizer, TFBertModel, BertConfig, TFBertForSequenceClassification
from transformers import glue_convert_examples_to_features
import pandas as pd
import tensorflow_datasets

In [33]:
train = pd.read_csv('/content/drive/My Drive/nlp-getting-started/train.csv')
test = pd.read_csv('/content/drive/My Drive/nlp-getting-started/test.csv')
submission = pd.read_csv("/content/drive/My Drive/nlp-getting-started/sample_submission.csv")

In [34]:
ids_with_target_error = [328,443,513,2619,3640,3900,4342,5781,6552,6554,6570,6701,6702,6729,6861,7226]
train.loc[train['id'].isin(ids_with_target_error),'target'] = 0

In [35]:
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
model = TFBertForSequenceClassification.from_pretrained('bert-base-cased')
data = tensorflow_datasets.load('glue/mrpc')

- This IS expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
INFO:absl:Overwrite dataset info from restored data version.
INFO:absl:Reusing dataset glue (/root/tensorflow_datasets/glue/mrpc/1.0.0)
INFO:absl:Constructing tf.data.Dataset for split None, from /root/tensorflow_datasets/glue/mrpc/1.0.0


In [46]:
train_dataset = glue_convert_examples_to_features(data['train'], tokenizer, max_length = 128, task = 'mrpc')
valid_dataset = glue_convert_examples_to_features(data['validation'], tokenizer, max_length = 128, task = 'mrpc')
train_dataset = train_dataset.shuffle(100).batch(32).repeat(2)
valid_dataset = valid_dataset.batch(64)

In [49]:
optimizer = tf.keras.optimizers.Adam(learning_rate = 3e-5, epsilon = 1e-8, clipnorm =1.0)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits = True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
model.compile(optimizer = optimizer, loss = loss, metrics = [metric])

In [ ]:
model.fit(train_dataset, epochs = 2, steps_per_epoch = 115, validation_data = valid_dataset, validation_steps = 7)